In [1]:
import math

import numpy as np
import plotly.express as px
import pandas as pd

from tasks.task1.main import generate_hilbert_matrix

In [2]:
# сумма всех элементов строки i без диагонального
def find_r_i(A: np.array, i: int):
    n = A.shape[0]
    r_i = 0
    for j in range(n):
        if j == i:
            continue
        r_i += abs(A[i, j])
    return r_i


def find_r_list(A: np.array):
    n = A.shape[0]
    r = []
    for i in range(n):
        r.append(find_r_i(A, i))
    return r


def check_gershgorin(A: np.array, eigen_values: list) -> bool:
    n = A.shape[0]
    r = find_r_list(A)

    for eigen_value in eigen_values:
        answer = False
        for i in range(n):
            if abs(eigen_value - A[i, i]) <= r[i]:
                answer = True
                break
        if not answer:
            return False
    return True

#
# def is_upper_triangular(A: np.array): return np.allclose(A, np.triu(A))
#
#
# def is_diagonal(A: np.array): return np.allclose(A, np.tril(np.triu(A)))

In [3]:
from math import sqrt


def find_sin_cos(A: np.array, i: int, j: int):
    x = -2 * A[i, j]
    y = A[i, i] - A[j, j]
    if y == 0:
        cos = 1 / sqrt(2)
        sin = 1 / sqrt(2)
    else:
        cos = math.sqrt(0.5 * (1 + abs(y) / math.sqrt(x ** 2 + y ** 2)))
        sin = np.sign(x * y) * abs(x) / (2 * cos * math.sqrt(x ** 2 + y ** 2))
    return sin, cos


def create_rotational_matrix(n, sin, cos, i, j) -> np.array:
    if i > j:
        matrix = np.eye(n)
        matrix[i, i] = cos
        matrix[i, j] = sin
        matrix[j, i] = -sin
        matrix[j, j] = cos
    else:
        matrix = np.eye(n)
        matrix[i, i] = cos
        matrix[i, j] = -sin
        matrix[j, i] = sin
        matrix[j, j] = cos
    return matrix


# for the case when i < j
def find_max_abs_non_diagonal_element_with_position(A: np.array):
    max_element = (-math.inf, (-1, -1))
    n = A.shape[0]
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            if abs(A[i, j]) > max_element[0]:
                max_element = (abs(A[i, j]), (i, j))
    print(max_element[0])
    return max_element[1]


class NextNonDiagonalElementFinder:
    def __init__(self, n):
        assert n >= 2, "n must be greater than 1"
        self.indices_list = []
        self.current_index_pair_pointer = 0

        for i in range(n):
            for j in range(n):
                if i != j:
                    self.indices_list.append((i, j))

    def get_next_index_pair(self):
        if self.current_index_pair_pointer == len(self.indices_list):
            self.current_index_pair_pointer = 0
        value = self.indices_list[self.current_index_pair_pointer]
        self.current_index_pair_pointer += 1
        return value

# strategy:
# 1 -> max upper non-diagonal
# 2 -> next numbered non-diagonal
def find_eig(A: np.array, strategy: int, epsilon = 10 ** 0):
    n = A.shape[0]
    R = A.copy()
    iteration_steps = 0
    next_index_pair_finder = NextNonDiagonalElementFinder(n)

    while len([r for r in find_r_list(R) if r >= epsilon]) > 0 and iteration_steps < 1000:
        if strategy == 1:
            i, j = find_max_abs_non_diagonal_element_with_position(R)
        else:
            i, j = next_index_pair_finder.get_next_index_pair()
        sin, cos = find_sin_cos(R, i, j)
        rotational_matrix = create_rotational_matrix(n, sin, cos, i, j)

        # print(iteration_steps)
        # print(f"R list: {find_r_list(R)}")
        print(f"R:\n{R}")
        # print(f"rotational {i} {j}:\n{rotational_matrix}")
        # print()

        R = rotational_matrix @ R @ rotational_matrix.T
        iteration_steps += 1
    assert check_gershgorin(A, list(sort(np.diag(R)))), "Gershgorin fails"
    return R, iteration_steps


def test(A: np.array, strategy: int, epsilon = 10 ** 0, info=False):
    exp_eigs = sort(np.linalg.eig(A)[0])
    R, iteration_steps = find_eig(A, strategy, epsilon)
    if info:
        print(f"Iteration steps: {iteration_steps}")
    act_eigs = sort(np.diag(R))
    if info:
        print(f"Expected: {exp_eigs}")
        print(f"Actual: {act_eigs}")
        print(f"Discrepancy: {[np.linalg.norm(expected - actual) for (expected, actual) in zip(exp_eigs, act_eigs)]}")
    return iteration_steps, act_eigs

In [5]:
A = np.array([
 [1, 2, 5],
 [2, 3, 2],
 [5, 2, 9]
])
i, j = 1, 0
sin, cos = find_sin_cos(A, i, j)
print(create_rotational_matrix(3, sin, cos, i, j))

[[ 0.85065081  0.52573111  0.        ]
 [-0.52573111  0.85065081  0.        ]
 [ 0.          0.          1.        ]]


In [95]:
A = np.array([[6, 5, 0],
              [5, 1, 4],
              [0, 4, 3]], float)
epsilon = 10 ** (-2)
test(A, 1, epsilon, True)

5.0
R:
[[6. 5. 0.]
 [5. 1. 4.]
 [0. 4. 3.]]
3.4026032334081595
R:
[[ 9.09016994e+00 -1.40921408e-16  2.10292445e+00]
 [-3.41124895e-16 -2.09016994e+00  3.40260323e+00]
 [ 2.10292445e+00  3.40260323e+00  3.00000000e+00]]
1.8803044280999428
R:
[[ 9.09016994e+00 -9.41672180e-01  1.88030443e+00]
 [-9.41672180e-01 -3.79422209e+00 -8.47255322e-16]
 [ 1.88030443e+00 -8.16618074e-16  4.70405215e+00]]
0.883157053919665
R:
[[ 9.78589142e+00 -8.83157054e-01  1.73414536e-16]
 [-8.83157054e-01 -3.79422209e+00  3.26772263e-01]
 [ 1.76694559e-16  3.26772263e-01  4.00833067e+00]]
1.7515604741808866
R:
[[ 9.61526614 -1.75156047  0.02111766]
 [-1.75156047 -3.62359681  0.32608918]
 [ 0.02111766  0.32608918  4.00833067]]
3.386566568037176
R:
[[ 8.94696935 -3.38656657  0.06300019]
 [-3.38656657 -2.95530002  0.32064168]
 [ 0.06300019  0.32064168  4.00833067]]
0.32608918469297216
R:
[[ 9.84308500e+00  1.01139689e-16 -2.11176563e-02]
 [ 1.33195972e-16 -3.85141567e+00  3.26089185e-01]
 [-2.11176563e-02  3.2608

(8, array([-3.86492142,  4.02175994,  9.84316148]))

In [51]:
def plot_iteration_steps_to_epsilon_degree(df):
    fig = px.line(df, x="epsilon degree", y="iteration steps", color="method")
    fig.show()

In [87]:
from numpy import sort

A = np.array([[1, 1, 0],
              [1, 2, 0],
              [0, 0, 3]], float)
epsilon = 10 ** (-4)
strategy = 1
test(A, strategy, epsilon, True)

Iteration steps: 1
Expected: [0.38196601 2.61803399 3.        ]
Actual: [0.38196601 2.61803399 3.        ]
Discrepancy: [0.0, 4.440892098500626e-16, 0.0]


(1, array([0.38196601, 2.61803399, 3.        ]))

In [96]:
from numpy import sort

A = np.array([[1, 1, 0],
              [1, 2, 0],
              [0, 0, 3]], float)
epsilon = 10 ** (-1)

data = []
for i in range(-10, -1):
    epsilon = 10 ** i
    iterations_number_1, _ = test(A, 1, epsilon, False)
    iterations_number_2, _ = test(A, 2, epsilon, False)
    data.append([i, iterations_number_1, "max"])
    data.append([i, iterations_number_2, "next"])
df = pd.DataFrame(data, columns=["epsilon degree", "iteration steps", "method"])
plot_iteration_steps_to_epsilon_degree(df)

1.0
R:
[[1. 1. 0.]
 [1. 2. 0.]
 [0. 0. 3.]]
R:
[[1. 1. 0.]
 [1. 2. 0.]
 [0. 0. 3.]]
1.0
R:
[[1. 1. 0.]
 [1. 2. 0.]
 [0. 0. 3.]]
R:
[[1. 1. 0.]
 [1. 2. 0.]
 [0. 0. 3.]]
1.0
R:
[[1. 1. 0.]
 [1. 2. 0.]
 [0. 0. 3.]]
R:
[[1. 1. 0.]
 [1. 2. 0.]
 [0. 0. 3.]]
1.0
R:
[[1. 1. 0.]
 [1. 2. 0.]
 [0. 0. 3.]]
R:
[[1. 1. 0.]
 [1. 2. 0.]
 [0. 0. 3.]]
1.0
R:
[[1. 1. 0.]
 [1. 2. 0.]
 [0. 0. 3.]]
R:
[[1. 1. 0.]
 [1. 2. 0.]
 [0. 0. 3.]]
1.0
R:
[[1. 1. 0.]
 [1. 2. 0.]
 [0. 0. 3.]]
R:
[[1. 1. 0.]
 [1. 2. 0.]
 [0. 0. 3.]]
1.0
R:
[[1. 1. 0.]
 [1. 2. 0.]
 [0. 0. 3.]]
R:
[[1. 1. 0.]
 [1. 2. 0.]
 [0. 0. 3.]]
1.0
R:
[[1. 1. 0.]
 [1. 2. 0.]
 [0. 0. 3.]]
R:
[[1. 1. 0.]
 [1. 2. 0.]
 [0. 0. 3.]]
1.0
R:
[[1. 1. 0.]
 [1. 2. 0.]
 [0. 0. 3.]]
R:
[[1. 1. 0.]
 [1. 2. 0.]
 [0. 0. 3.]]


In [97]:
A = np.array([[6, 5, 0],
              [5, 1, 4],
              [0, 4, 3]], float)
data = []
for i in range(-3, 1):
    epsilon = 10 ** i
    iterations_number_1, _ = test(A, 1, epsilon, False)
    iterations_number_2, _ = test(A, 2, epsilon, False)
    data.append([i, iterations_number_1, "max"])
    data.append([i, iterations_number_2, "next"])
df = pd.DataFrame(data, columns=["epsilon degree", "iteration steps", "method"])
plot_iteration_steps_to_epsilon_degree(df)

5.0
R:
[[6. 5. 0.]
 [5. 1. 4.]
 [0. 4. 3.]]
3.4026032334081595
R:
[[ 9.09016994e+00 -1.40921408e-16  2.10292445e+00]
 [-3.41124895e-16 -2.09016994e+00  3.40260323e+00]
 [ 2.10292445e+00  3.40260323e+00  3.00000000e+00]]
1.8803044280999428
R:
[[ 9.09016994e+00 -9.41672180e-01  1.88030443e+00]
 [-9.41672180e-01 -3.79422209e+00 -8.47255322e-16]
 [ 1.88030443e+00 -8.16618074e-16  4.70405215e+00]]
0.883157053919665
R:
[[ 9.78589142e+00 -8.83157054e-01  1.73414536e-16]
 [-8.83157054e-01 -3.79422209e+00  3.26772263e-01]
 [ 1.76694559e-16  3.26772263e-01  4.00833067e+00]]
1.7515604741808866
R:
[[ 9.61526614 -1.75156047  0.02111766]
 [-1.75156047 -3.62359681  0.32608918]
 [ 0.02111766  0.32608918  4.00833067]]
3.386566568037176
R:
[[ 8.94696935 -3.38656657  0.06300019]
 [-3.38656657 -2.95530002  0.32064168]
 [ 0.06300019  0.32064168  4.00833067]]
0.32608918469297216
R:
[[ 9.84308500e+00  1.01139689e-16 -2.11176563e-02]
 [ 1.33195972e-16 -3.85141567e+00  3.26089185e-01]
 [-2.11176563e-02  3.2608

In [100]:
n = 5
A = generate_hilbert_matrix(n)
data = []
for i in range(-10, 1):
    epsilon = 10 ** i
    iterations_number_1, _ = test(A, 1, epsilon, False)
    iterations_number_2, _ = test(A, 2, epsilon, False)
    data.append([i, iterations_number_1, "max"])
    data.append([i, iterations_number_2, "next"])
df = pd.DataFrame(data, columns=["epsilon degree", "iteration steps", "method"])
plot_iteration_steps_to_epsilon_degree(df)

0.5
R:
[[1.         0.5        0.33333333 0.25       0.2       ]
 [0.5        0.33333333 0.25       0.2        0.16666667]
 [0.33333333 0.25       0.2        0.16666667 0.14285714]
 [0.25       0.2        0.16666667 0.14285714 0.125     ]
 [0.2        0.16666667 0.14285714 0.125      0.11111111]]
0.411856014305654
R:
[[ 1.26759188e+00 -8.67013176e-18  4.11856014e-01  3.14790235e-01
   2.54977907e-01]
 [-2.93681144e-17  6.57414541e-02  6.31326745e-02  5.83704384e-02
   5.25741814e-02]
 [ 4.11856014e-01  6.31326745e-02  2.00000000e-01  1.66666667e-01
   1.42857143e-01]
 [ 3.14790235e-01  5.83704384e-02  1.66666667e-01  1.42857143e-01
   1.25000000e-01]
 [ 2.54977907e-01  5.25741814e-02  1.42857143e-01  1.25000000e-01
   1.11111111e-01]]
0.35173259812823865
R:
[[1.40800917e+00 2.03728061e-02 1.37997960e-16 3.51732598e-01
  2.87436825e-01]
 [2.03728061e-02 6.57414541e-02 5.97551953e-02 5.83704384e-02
  5.25741814e-02]
 [3.27160220e-17 5.97551953e-02 5.95827080e-02 5.61680435e-02
  5.293362